# Constantes y Funciones Auxiliares

In [8]:
# Semilla
SEED = 333

# Preparamos el lematizado
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

# Lematizar un string
import re
def stemming_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [porter_stemmer.stem(word) for word in words]
    return words

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

## Columnas de guardado para los algortimos 
COLUMNS = ['mean_fit_time','std_fit_time','mean_test_neg_log_loss','std_test_neg_log_loss','rank_test_neg_log_loss',
           'mean_test_accuracy','rank_test_accuracy',
           'mean_test_f1_macro','rank_test_f1_macro',
           'mean_test_roc_auc_ovr','rank_test_roc_auc_ovr']

# Funcion de guardado de resultados que es un subconjunto de cv_results. 
# Guarda los resultados de los parametros del algoritmo y las metricas que le pasamos como parametro.
def save_results(rs,params_to_evaluate,columns=COLUMNS):
    aux = pd.DataFrame(rs.cv_results_)
    gs_res = pd.DataFrame()
    for col in params_to_evaluate:
        gs_res[col] = aux[col]
    for col in columns:
        gs_res[col] = aux[col]
    return gs_res

# Carga

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

train_variants_df = pd.read_csv(r"C:\Users\Junio\Libretas\data-c\training_variants", engine='python')
train_txt_df = pd.read_csv(r"C:\Users\Junio\Libretas\data-c/training_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
train_txt_df['Class'] = train_variants_df['Class']
train_txt_df.sample(10,random_state=SEED)

,ID,Text,Class
3198,3198,Inactivation of Ras GTPase activating proteins...,1
2161,2161,The PTEN (phosphatase and tensin homolog) phos...,4
1083,1083,"EZH2, the catalytic subunit of the PRC2 comple...",9
113,113,Endometrial cancer is the most common gynecolo...,4
885,885,Purpose: Platelet-derived growth factor recept...,7
3210,3210,Hereditary predisposition to retinoblastoma is...,1
153,153,Many studies have reported the EGFR mutations ...,2
2312,2312,A systematic characterization of the genetic a...,7
1107,1107,"Multiple endocrine neoplasia type 1 (MEN1, OM...",4
1648,1648,"In acute myeloid leukemia (AML), two clusters ...",7


# Preparacion del Dataframe

In [9]:
# Inicializamos el dataframe que vamos a utilizar
W = pd.DataFrame()

# Añadimos una columna que nos indica el tamaño del texto de cada instancia
W['Text_count']  = train_txt_df["Text"].apply(lambda x: len(str(x).split()))

# Copiamos la clase y el texto
W['Class'] = train_txt_df['Class'].copy()
W['Text'] = train_txt_df["Text"].copy()

# Nos quedamos con las instancias que no tengan el texto nulo
W = W[W['Text_count']!=1]

# Mostramos el dataframe
W.sample(10,random_state=SEED)

,Text_count,Class,Text
3198,3343,1,Inactivation of Ras GTPase activating proteins...
2084,3005,7,MYD88 L265P is a somatic mutation that has bee...
3185,11198,6,We describe the case of a patient presenting w...
113,19533,4,Endometrial cancer is the most common gynecolo...
3133,15135,7,Transforming mutations in NRAS and KRAS are th...
1110,14851,1,Abstract Fanconi anemia is characterized by c...
153,3837,2,Many studies have reported the EGFR mutations ...
1242,65740,7,The development of array comparative genomic h...
2768,6895,7,Over 30 mutations of the B-RAF gene associated...
1045,10154,1,Tuberous sclerosis (TSC) is an autosomal domin...


# Preparando la Clasificacion 

In [10]:
# Separacion training/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(W['Text'], W['Class'], test_size=0.2, random_state=SEED)

In [11]:
# Preparando stop_words
from wordcloud import STOPWORDS

# Capturando las palabras que no se encuentran en STOPWORDS por ser una contraccion de la palabra original (salida de stemming_tokenizer)
contract_words = {'abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 
              'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 
              'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 
              'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 
              'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 
              'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 
              'someth', 'sometim', 'somewher', 'themselv', 'thenc', 
              'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 
              'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 
              'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'}

l2 = {'anywh', 'aren', 'becau', 'couldn', 'd', 'didn', 'doe', 
      'doesn', 'don', 'el', 'elsewh', 'everywh', 'hadn', 'hasn', 
      'haven', 'ind', 'isn', 'let', 'll', 'm', 'mustn', 
      'otherwi', 'plea', 're', 's', 'shan', 'shouldn', 
      'somewh', 't', 've', 'wasn', 'weren', 'won', 'wouldn'}

custom_words = {"fig", "figure", "et", "al", "al.", "also",
                "data", "analyze", "study", "table", "using",
                "method", "result", "conclusion", "author", 
                "find", "found", "show", "casita","non","name","image",
                'analyz', 'conclus', 'figur','conclu', 'imag', 'studi', 'tabl', 'use'}
# Unimos ambas listas
stop_words = STOPWORDS.union(contract_words).union(l2)

from sklearn.pipeline import Pipeline
def create_pipeline(clf,ngram_range=(1,1)):
    return Pipeline([('tfidf', TfidfVectorizer(analyzer="word", tokenizer=stemming_tokenizer,stop_words= stop_words,ngram_range=ngram_range)),
                         ('clf', clf)])


# Validacion Cruzada Stratificada(n_splits=3):
from sklearn.model_selection import StratifiedKFold
CV = StratifiedKFold(n_splits=3, random_state=SEED, shuffle=True)

from sklearn.model_selection import RandomizedSearchCV
def create_rscv(pipeline,params,n_iterations = 10,scoring = ["neg_log_loss","accuracy","f1_macro","roc_auc_ovr"],cv = CV):
    return RandomizedSearchCV(
            pipeline,
            params,
            n_iter = n_iterations,
            verbose = 1,
            random_state = SEED,
            cv = cv,
            n_jobs = -1,
            scoring = scoring,
            refit = "neg_log_loss" 
            )
# Importamos la metrica principal de evaluacion
from sklearn import metrics

# Dataframe de guardado del test
df_results = pd.DataFrame(columns = ["clf","log_loss","accuracy","f1-macro","ROC"])

# Funcion de guardado del resultado del test
def add_res(clf,name,X_test = X_test):   
    # Guardamos las predicciones
    y_predict_proba = clf.predict_proba(X_test)
    y_pred = clf.predict(X_test)
    
    # Guardamos los resultados de las distintas metricas
    log_loss = metrics.log_loss(y_test,y_predict_proba)
    acc = metrics.accuracy_score(y_test,y_pred)
    f1 = metrics.f1_score(y_test, y_pred, average='macro')
    roc = metrics.roc_auc_score(y_test,y_predict_proba,multi_class='ovr')
    
    # Actualizamos el dataframe
    df_results.loc[len(df_results)]=[name,log_loss,acc,f1,roc]

## Clasificacion

## Naive Bayes Multinomial

In [12]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
parameters = {
    'tfidf__ngram_range': ((1, 1),(2, 2),(1,2)),
    'tfidf__use_idf': (True, False),
    'tfidf__max_features': (1000,2000,3000,4000,5000,6000,7000,8000,9000,None),
    'tfidf__norm': ('l1', 'l2')
 }
pipeline = create_pipeline(clf)
rs_NB_M = create_rscv(pipeline,parameters)

In [13]:
rs_NB_M.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 238.3min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=333, shuffle=True),
                   estimator=Pipeline(steps=[('tfidf',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'abov',
                                                                          'above',
                                                                          'after',
                                                                          'afterward',
                                                                          'again',
                                                                          'against',
                                                                          'all',
                                                                          'alon',
                                  

In [14]:
params_to_evaluate = ["param_tfidf__max_features","param_tfidf__ngram_range","param_tfidf__use_idf",
                      "param_tfidf__norm"]
NB_M_res = save_results(rs_NB_M,params_to_evaluate)
NB_M_res.sort_values(by='mean_test_neg_log_loss',ascending=False).head(5)

,param_tfidf__max_features,param_tfidf__ngram_range,param_tfidf__use_idf,param_tfidf__norm,mean_fit_time,std_fit_time,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,mean_test_accuracy,rank_test_accuracy,mean_test_f1_macro,rank_test_f1_macro,mean_test_roc_auc_ovr,rank_test_roc_auc_ovr
7,1000,"(2, 2)",True,l2,842.279479,6.601352,-1.304260,0.010437,1,0.535445,2,0.328068,1,0.830213,1
8,5000,"(1, 1)",True,l2,810.378766,6.940436,-1.460152,0.040744,2,0.538839,1,0.297690,2,0.801549,2
0,3000,"(2, 2)",True,l1,841.407810,10.784107,-1.778819,0.003577,3,0.299397,3,0.066202,3,0.745292,7
4,5000,"(1, 1)",True,l1,1016.755881,8.411149,-1.793446,0.003256,4,0.282428,4,0.048940,4,0.782798,4
9,6000,"(2, 2)",True,l1,751.402773,71.727727,-1.797094,0.003284,5,0.282428,4,0.048940,4,0.761074,6


In [15]:
# Exportamos los resultados
NB_M_res.to_csv('NB_M.csv')

# Testing
add_res(rs_NB_M,'NB_M')

## Complement Naive Bayes

In [16]:
from sklearn.naive_bayes import ComplementNB
clf2 = ComplementNB()
parameters = {
    'tfidf__ngram_range': ((1, 1),(2, 2),(1,2)),
    'tfidf__use_idf': (True, False),
    'tfidf__max_features': (1000,2000,3000,4000,5000,6000,7000,8000,9000,None),
    'tfidf__norm': ('l1', 'l2')
 }
pipeline2 = create_pipeline(clf2)
rs_NB_C = create_rscv(pipeline2,parameters)

In [17]:
rs_NB_C.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 257.5min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=333, shuffle=True),
                   estimator=Pipeline(steps=[('tfidf',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'abov',
                                                                          'above',
                                                                          'after',
                                                                          'afterward',
                                                                          'again',
                                                                          'against',
                                                                          'all',
                                                                          'alon',
                                  

In [18]:
params_to_evaluate = ["param_tfidf__max_features","param_tfidf__ngram_range","param_tfidf__use_idf",
                      "param_tfidf__norm"]
NB_C_res = save_results(rs_NB_C,params_to_evaluate)
NB_C_res.sort_values(by='mean_test_neg_log_loss',ascending=False).head(5)

,param_tfidf__max_features,param_tfidf__ngram_range,param_tfidf__use_idf,param_tfidf__norm,mean_fit_time,std_fit_time,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,mean_test_accuracy,rank_test_accuracy,mean_test_f1_macro,rank_test_f1_macro,mean_test_roc_auc_ovr,rank_test_roc_auc_ovr
8,5000,"(1, 1)",True,l2,903.699222,8.549372,-1.583684,0.006781,1,0.595023,1,0.451671,1,0.851182,1
7,1000,"(2, 2)",True,l2,1051.410870,6.113063,-1.729723,0.002472,2,0.533183,4,0.333714,5,0.834278,4
0,3000,"(2, 2)",True,l1,1049.933705,5.517039,-2.151305,0.000111,3,0.552036,3,0.394429,3,0.826113,7
6,8000,"(1, 1)",False,l1,1035.710302,30.535763,-2.153315,0.000049,4,0.362745,9,0.109663,9,0.805810,9
2,9000,"(1, 1)",True,l1,1012.452565,3.826718,-2.155405,0.000202,5,0.519608,7,0.286583,7,0.842265,2


In [19]:
# Exportamos los resultados
NB_C_res.to_csv('NB_C.csv')

# Testing
add_res(rs_NB_C,'NB_C')

## KNN

In [20]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
parameters = {
    'tfidf__ngram_range': ((1, 1),(2, 2),(1,2)),
    'tfidf__use_idf': (True, False),
    'tfidf__max_features': (1000,2000,3000,4000,5000,6000,7000,8000,9000,None),
    'tfidf__norm': ('l1', 'l2'),
    'clf__n_neighbors': tuple(range(1,60,2))
 }
pipeline5 = create_pipeline(knn)
rs_KNN = create_rscv(pipeline5,parameters)

In [21]:
rs_KNN.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 232.6min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=333, shuffle=True),
                   estimator=Pipeline(steps=[('tfidf',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'abov',
                                                                          'above',
                                                                          'after',
                                                                          'afterward',
                                                                          'again',
                                                                          'against',
                                                                          'all',
                                                                          'alon',
                                  

In [22]:
params_to_evaluate = ["param_tfidf__max_features","param_tfidf__ngram_range","param_tfidf__use_idf",
                      "param_tfidf__norm","param_clf__n_neighbors"]
KNN_res = save_results(rs_KNN,params_to_evaluate)
KNN_res.sort_values(by='mean_test_neg_log_loss',ascending=False).head(5)

,param_tfidf__max_features,param_tfidf__ngram_range,param_tfidf__use_idf,param_tfidf__norm,param_clf__n_neighbors,mean_fit_time,std_fit_time,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,mean_test_accuracy,rank_test_accuracy,mean_test_f1_macro,rank_test_f1_macro,mean_test_roc_auc_ovr,rank_test_roc_auc_ovr
8,5000,"(1, 2)",False,l2,59,860.185606,5.795864,-1.665384,0.053852,1,0.478884,7,0.275628,8,0.818245,6
4,7000,"(2, 2)",True,l2,57,1000.285799,9.171289,-1.775546,0.116579,2,0.476621,9,0.258317,9,0.829799,3
3,5000,"(1, 1)",False,l2,33,937.439335,7.433932,-1.973448,0.090360,3,0.493967,6,0.304978,6,0.831315,2
7,4000,"(1, 1)",True,l1,59,817.203187,5.248160,-2.019685,0.083152,4,0.430241,10,0.230764,10,0.808625,8
2,1000,"(1, 1)",False,l2,25,854.041332,45.037448,-2.188570,0.133254,5,0.519985,4,0.359224,5,0.833732,1


In [23]:
# Exportamos los resultados
KNN_res.to_csv('KNN.csv')

# Testing
add_res(rs_KNN,'KNN')

## Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier
clf3 = RandomForestClassifier(random_state=SEED)
parameters = {
    'tfidf__ngram_range': ((1, 1),(2, 2),(1,2)),
    'tfidf__use_idf': (True, False),
    'tfidf__max_features': (1000,2000,3000,4000,5000,6000,7000,8000,9000,None),
    'tfidf__norm': ('l1', 'l2'),
    'clf__criterion':('giny','entropy')
 }
pipeline3 = create_pipeline(clf3)
rs_RF = create_rscv(pipeline3,parameters)

In [25]:
rs_RF.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 163.1min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=333, shuffle=True),
                   estimator=Pipeline(steps=[('tfidf',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'abov',
                                                                          'above',
                                                                          'after',
                                                                          'afterward',
                                                                          'again',
                                                                          'against',
                                                                          'all',
                                                                          'alon',
                                  

In [26]:
params_to_evaluate = ["param_tfidf__max_features","param_tfidf__ngram_range","param_tfidf__use_idf",
                      "param_tfidf__norm","param_clf__criterion"]
RF_res = save_results(rs_RF,params_to_evaluate)
RF_res.sort_values(by='mean_test_neg_log_loss',ascending=False).head(5)

,param_tfidf__max_features,param_tfidf__ngram_range,param_tfidf__use_idf,param_tfidf__norm,param_clf__criterion,mean_fit_time,std_fit_time,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,mean_test_accuracy,rank_test_accuracy,mean_test_f1_macro,rank_test_f1_macro,mean_test_roc_auc_ovr,rank_test_roc_auc_ovr
9,1000,"(1, 2)",True,l2,entropy,782.512933,66.025566,-1.664640,0.077941,1,0.635370,2,0.502398,2,0.873852,2
6,5000,"(1, 1)",True,l2,entropy,855.589561,6.132000,-1.667135,0.037609,2,0.631599,3,0.500690,4,0.862489,5
8,4000,"(1, 1)",True,l1,entropy,861.420969,7.764854,-1.671802,0.110835,3,0.636124,1,0.504538,1,0.870616,3
1,2000,"(1, 1)",False,l2,entropy,850.762133,3.939272,-1.691484,0.084461,4,0.628205,5,0.500036,5,0.869830,4
3,9000,"(1, 1)",False,l1,entropy,880.478687,9.337633,-1.711282,0.121392,5,0.631599,3,0.501721,3,0.876962,1


In [27]:
# Exportamos los resultados
RF_res.to_csv('RF.csv')

# Testing
add_res(rs_RF,'RF')

## SVC

In [28]:
from sklearn.svm import SVC
clf4 = SVC(probability=True)
parameters = {
    'tfidf__ngram_range': ((1, 1),(2, 2),(1,2)),
    'tfidf__use_idf': (True, False),
    'tfidf__max_features': (1000,2000,3000,4000,5000,6000,7000,8000,9000,None),
    'tfidf__norm': ('l1', 'l2'),
    'clf__kernel':('linear','rbf','sigmoid','poly')
 }
pipeline4 = create_pipeline(clf4)
rs_SVC = create_rscv(pipeline4,parameters)

In [29]:
rs_SVC.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 279.2min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=333, shuffle=True),
                   estimator=Pipeline(steps=[('tfidf',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'abov',
                                                                          'above',
                                                                          'after',
                                                                          'afterward',
                                                                          'again',
                                                                          'against',
                                                                          'all',
                                                                          'alon',
                                  

In [30]:
params_to_evaluate = ["param_tfidf__max_features","param_tfidf__use_idf","param_tfidf__ngram_range","param_tfidf__norm","param_clf__kernel"]
SVC_res = save_results(rs_SVC,params_to_evaluate)
SVC_res.sort_values(by='mean_test_neg_log_loss',ascending=False).head(5)

,param_tfidf__max_features,param_tfidf__use_idf,param_tfidf__ngram_range,param_tfidf__norm,param_clf__kernel,mean_fit_time,std_fit_time,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,mean_test_accuracy,rank_test_accuracy,mean_test_f1_macro,rank_test_f1_macro,mean_test_roc_auc_ovr,rank_test_roc_auc_ovr
1,7000,True,"(1, 1)",l2,rbf,1178.656833,12.020918,-1.059184,0.006116,1,0.625189,1,0.497468,1,0.893184,1
3,5000,True,"(2, 2)",l2,rbf,1036.365253,11.086397,-1.059470,0.009264,2,0.624057,2,0.492285,3,0.892005,2
6,9000,True,"(1, 2)",l2,poly,1429.277459,9.088498,-1.078076,0.004070,3,0.615762,3,0.493106,2,0.890569,3
7,5000,False,"(1, 1)",l2,linear,1033.692731,10.363156,-1.086370,0.022727,4,0.612745,4,0.467052,4,0.880357,4
8,7000,True,"(1, 1)",l2,sigmoid,1079.909835,9.782600,-1.109774,0.007586,5,0.602187,5,0.448780,5,0.879036,5


In [31]:
# Exportamos los resultados
SVC_res.to_csv('SVC.csv')

# Testing
add_res(rs_SVC,'SVC')

In [32]:
# Exportamos resultados totales
df_results.to_csv('df_rs.csv')
df_results

,clf,log_loss,accuracy,f1-macro,ROC
0,NB_M,1.339528,0.552711,0.336474,0.802195
1,NB_C,1.565477,0.591867,0.427967,0.838728
2,KNN,1.669148,0.504518,0.282924,0.829964
3,RF,1.460784,0.612952,0.531519,0.883515
4,SVC,1.033469,0.637048,0.503059,0.890707
